In [1]:
import pandas as pd
import numpy as np

In [2]:
sampel = pd.read_csv('D:/Belajar python/mlc_dataslayer_revisi/sample_submission.csv')
test = pd.read_csv('D:/Belajar python/mlc_dataslayer_revisi/test.csv')
df = pd.read_csv('D:/Belajar python/mlc_dataslayer_revisi/train.csv')

In [3]:
df.head()

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km)
0,1,MITSU,SUV - SMALL,1.5,4.0,AV8,X,8.40 km/L,0.72 L/10km,0.98 L/10km,208
1,2,TOYOTI,PICKUP TRUCK - SMALL,not-available,6.0,A5,X,7.25 km per L,9.70 L/100km,11.96 L/100km,325
2,3,MATSUDA,COMPACT,2.0,4.0,AS6,X,9.80 km/L,38.70 mpg Imp.,31.76 mpg Imp.,170
3,4,CHEVO,VAN - PASSENGER,unknown,8.0,A6,X,1.73 L/10km,11.70 liters per 100 km,14.78 liters per 100 km,362
4,5,TOYOTI,COMPACT,1.8,4.0,M6,X,8.10 L/100km,35.76 mpg Imp.,8.01 liters per 100 km,180


In [5]:
df.isna().sum()

Id                          0
Make                        0
Vehicle Class             539
Engine Size(L)           1393
Cylinders                1198
Transmission              410
Fuel Type                 546
Fuel Consumption City    1568
Fuel Consumption Hwy     1701
Fuel Consumption Comb    1674
CO2 Emissions(g/km)         0
dtype: int64

In [6]:
def remove_units(x):
    to_remove = ['liters per 100 km', 'L/10km', 'km per L', 'L/100km', 'MPG (AS)',
       'mpg Imp.', 'L/100 km', 'km/L']
    for i in to_remove:
        x = str(x).replace(i,'')
    return x

def convert_units(x):
    if x in ['km/L', 'km per L']:
        return 'km/L'
    elif x in ['L/100 km', 'L/100km', 'liters per 100 km']:
        return 'L/100km'
    else:
        return x

def convert_fuel_city(row):
    if row['Fuel Consumption City units'] == 'L/10km':
        return row['Fuel Consumption City'] * 10
    elif row['Fuel Consumption City units'] == 'MPG (AS)':
        return 235.215 / row['Fuel Consumption City']
    elif row['Fuel Consumption City units'] == 'mpg Imp.':
        return 282.481 / row['Fuel Consumption City']
    elif row['Fuel Consumption City units'] == 'km/L':
        return 100 / row['Fuel Consumption City']
    else:
        return row['Fuel Consumption City']
    
def convert_fuel_hwy(row):
    if row['Fuel Consumption Hwy units'] == 'L/10km':
        return row['Fuel Consumption Hwy'] * 10
    elif row['Fuel Consumption Hwy units'] == 'MPG (AS)':
        return 235.215 / row['Fuel Consumption Hwy']
    elif row['Fuel Consumption Hwy units'] == 'mpg Imp.':
        return 282.481 / row['Fuel Consumption Hwy']
    elif row['Fuel Consumption Hwy units'] == 'km/L':
        return 100 / row['Fuel Consumption Hwy']
    else:
        return row['Fuel Consumption Hwy']

def convert_fuel_comb(row):
    if row['Fuel Consumption Comb units'] == 'L/10km':
        return row['Fuel Consumption Comb'] * 10
    elif row['Fuel Consumption Comb units'] == 'MPG (AS)':
        return 235.215 / row['Fuel Consumption Comb']
    elif row['Fuel Consumption Comb units'] == 'mpg Imp.':
        return 282.481 / row['Fuel Consumption Comb']
    elif row['Fuel Consumption Comb units'] == 'km/L':
        return 100 / row['Fuel Consumption Comb']
    else:
        return row['Fuel Consumption Comb']
    
def data_preprocessing(data1):
    data = data1.copy()
    data['Fuel Consumption City'] = data['Fuel Consumption City'].map(lambda x: np.nan if x in ['0','not-available','-9999','-1','zero','9999'] else x)
    data['Fuel Consumption Comb'] = data['Fuel Consumption Comb'].map(lambda x: np.nan if x in ['0','not-available','-9999','-1','zero','9999'] else x)
    data['Fuel Consumption Hwy'] = data['Fuel Consumption Hwy'].map(lambda x: np.nan if x in ['0','not-available','-9999','-1','zero','9999'] else x)

    data['Fuel Consumption City units'] = data['Fuel Consumption City'].map(lambda x: str(x)[5:] if pd.isnull(x) == False else x)
    data['Fuel Consumption Hwy units'] = data['Fuel Consumption Hwy'].map(lambda x: str(x)[5:] if pd.isnull(x) == False else x)
    data['Fuel Consumption Comb units'] = data['Fuel Consumption Comb'].map(lambda x: str(x)[5:] if pd.isnull(x) == False else x)

    # City
    for i in range(7):
        data['Fuel Consumption City units'] = data['Fuel Consumption City units'].map(lambda x: str(x)[1:] if str(x)[0].isalpha() == False else x)

    # HWY
    for i in range(5):
        data['Fuel Consumption Hwy units'] = data['Fuel Consumption Hwy units'].map(lambda x: str(x)[1:] if str(x)[0].isalpha() == False else x)

    # Comb
    for i in range(5):
        data['Fuel Consumption Comb units'] = data['Fuel Consumption Comb units'].map(lambda x: str(x)[1:] if str(x)[0].isalpha() == False else x)

    data['Fuel Consumption City'] = data['Fuel Consumption City'].apply(remove_units).astype(float)
    data['Fuel Consumption Hwy'] = data['Fuel Consumption Hwy'].apply(remove_units).astype(float)
    data['Fuel Consumption Comb'] = data['Fuel Consumption Comb'].apply(remove_units).astype(float)

    data['Fuel Consumption City units'] = data['Fuel Consumption City units'].apply(convert_units)
    data['Fuel Consumption Hwy units'] = data['Fuel Consumption Hwy units'].apply(convert_units)
    data['Fuel Consumption Comb units'] = data['Fuel Consumption Comb units'].apply(convert_units)

    data['Fuel Consumption City'] = data.apply(convert_fuel_city, axis=1)
    data['Fuel Consumption Hwy'] = data.apply(convert_fuel_hwy, axis=1)
    data['Fuel Consumption Comb'] = data.apply(convert_fuel_comb, axis=1)

    data['Transmission'] = data['Transmission'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing','unestablished',
                                                                              'unspecified','na','not-available','-1'] else x)
    data['Cylinders'] = data['Cylinders'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing','unestablished',
                                                                        'unspecified','na','not-available','-1'] else x)
    data['Engine Size(L)'] = data['Engine Size(L)'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing',
                                                                                  'unestablished','unspecified','na','not-available','-1'] else x)
    data['Vehicle Class'] = data['Vehicle Class'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing',
                                                                                'unestablished','unspecified','na','not-available','-1'] else x)
    data['Fuel Type'] = data['Fuel Type'].map(lambda x: np.nan if x in ['not-recorded','unknown','missing',
                                                                        'unestablished','unspecified','na','not-available','-1'] else x)

    data['Cylinders'] = data['Cylinders'].astype(float)
    data['Engine Size(L)'] = data['Engine Size(L)'].astype(float)

    data.drop(['Fuel Consumption City units','Fuel Consumption Hwy units','Fuel Consumption Comb units'], axis=1, inplace=True)
    data = data.rename(columns={'CO2 Emissions(g/km)':'CO2_Emissions', 'Engine Size(L)':'Engine_Size', 'Fuel Consumption City':'Fuel_Consumption_City', 
                                'Fuel Consumption Hwy':'Fuel_Consumption_Hwy', 'Fuel Consumption Comb':'Fuel_Consumption_Comb', 
                                'Fuel Type':'Fuel_Type', 'Vehicle Class':'Vehicle_Class'})

    return data

In [8]:
df.head()

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km)
0,1,MITSU,SUV - SMALL,1.5,4.0,AV8,X,8.40 km/L,0.72 L/10km,0.98 L/10km,208
1,2,TOYOTI,PICKUP TRUCK - SMALL,not-available,6.0,A5,X,7.25 km per L,9.70 L/100km,11.96 L/100km,325
2,3,MATSUDA,COMPACT,2.0,4.0,AS6,X,9.80 km/L,38.70 mpg Imp.,31.76 mpg Imp.,170
3,4,CHEVO,VAN - PASSENGER,unknown,8.0,A6,X,1.73 L/10km,11.70 liters per 100 km,14.78 liters per 100 km,362
4,5,TOYOTI,COMPACT,1.8,4.0,M6,X,8.10 L/100km,35.76 mpg Imp.,8.01 liters per 100 km,180


In [7]:
data_clean = data_preprocessing(df)
data_clean.head()

,Id,Make,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Type,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,CO2_Emissions
0,1,MITSU,SUV - SMALL,1.5,4.0,AV8,X,11.904762,7.200000,9.800000,208
1,2,TOYOTI,PICKUP TRUCK - SMALL,NaN,6.0,A5,X,13.793103,9.700000,11.960000,325
2,3,MATSUDA,COMPACT,2.0,4.0,AS6,X,10.204082,7.299251,8.894238,170
3,4,CHEVO,VAN - PASSENGER,NaN,8.0,A6,X,17.300000,11.700000,14.780000,362
4,5,TOYOTI,COMPACT,1.8,4.0,M6,X,8.100000,7.899357,8.010000,180


In [10]:
data_clean.isna().sum()

Id                          0
Make                        0
Vehicle_Class            2051
Engine_Size              5264
Cylinders                4523
Transmission             1483
Fuel_Type                1942
Fuel_Consumption_City    2325
Fuel_Consumption_Hwy     2563
Fuel_Consumption_Comb    2552
CO2_Emissions               0
dtype: int64

In [11]:
test_clean = data_preprocessing(test)

In [12]:
data_clean.to_csv('D:/Belajar python/mlc_dataslayer_revisi/train_clean.csv', index=False)
test_clean.to_csv('D:/Belajar python/mlc_dataslayer_revisi/test_clean.csv', index=False)